In [ ]:
#Mohammad Mahdi Elyasi 
#403123091
#Problem 7 Serie 3
#Optical Network Communication
import numpy as np
import cvxpy as cp

# Topology adjacency matrix (undirected)
G = np.array([
    [0, 1, 1, 0, 1],  # Node 1
    [1, 0, 0, 0, 1],  # Node 2
    [1, 0, 0, 1, 0],  # Node 3
    [0, 0, 1, 0, 0],  # Node 4
    [1, 1, 0, 0, 0]   # Node 5
])

# Demand matrix (from image)
D = np.array([
    [0, 1, 1, 1, 0],
    [1, 0, 0, 1, 0],
    [1, 0, 0, 1, 1],
    [1, 1, 1, 0, 0],
    [0, 0, 1, 0, 0]
])

n = G.shape[0]

# Compute H-hop reachability matrix from topology G
def compute_reachability(G, H):
    R = np.zeros_like(G)
    A_power = np.identity(n)
    for h in range(1, H+1):
        A_power = np.dot(A_power, G)
        R = np.logical_or(R, A_power > 0)
    np.fill_diagonal(R, 0)
    return R.astype(int)

# ILP model
def solve_ILP(H):
    R = compute_reachability(G, H)

    # Binary decision variables: x[i][j] = 1 if demand D[i][j] is served
    x = cp.Variable((n, n), boolean=True)

    constraints = []

    # Only allow serving demand if:
    # 1. There is a demand (D[i][j] == 1)
    # 2. i and j are reachable within H hops (R[i][j] == 1)
    for i in range(n):
        for j in range(n):
            constraints.append(x[i, j] <= D[i, j])
            constraints.append(x[i, j] <= R[i, j])

    # Objective: maximize total served demands
    objective = cp.Maximize(cp.sum(x))

    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.GLPK_MI)

    print(f"\n=== H = {H} ===")
    print("Total Served Requests:", int(problem.value))
    print("Served Demands Matrix x:")
    print(np.round(x.value).astype(int))

# Solve for H = 1 and H = 2
solve_ILP(H=1)
solve_ILP(H=2)



=== H = 1 ===
Total Served Requests: 6
Served Demands Matrix x:
[[0 1 1 0 0]
 [1 0 0 0 0]
 [1 0 0 1 0]
 [0 0 1 0 0]
 [0 0 0 0 0]]

=== H = 2 ===
Total Served Requests: 10
Served Demands Matrix x:
[[0 1 1 1 0]
 [1 0 0 0 0]
 [1 0 0 1 1]
 [1 0 1 0 0]
 [0 0 1 0 0]]
